In [62]:
#pip install langchain-groq
#pip install environs
#pip install lanchain
# !pip -q install langchain  tiktoken chromadb pypdf sentence_transformers InstructorEmbedding



In [ ]:
from environs import Env

# Initialize environs
env = Env()
env.read_env(".env")

# Access the API key
api_key = env("GROQ_API_KEY")




In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [3]:
chat = ChatGroq(temperature=0.4, top_p=0.8,model_name="mixtral-8x7b-32768")

c:\Users\PRINCE SIDDHARTH\OneDrive\Desktop\rag_proj2\.venv\Lib\site-packages\langchain_groq\chat_models.py:362: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


In [4]:
from langchain.prompts import PromptTemplate

template = """ You are a helpful assistant.Always ready to help you with user questions.
Answer the question fully based on the context below. If you can't 
answer the question, reply "I don't know" .You dont hallucinate and you are fully reliable .

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

' You are a helpful assistant.Always ready to help you with user questions.\nAnswer the question fully based on the context below. If you can\'t \nanswer the question, reply "I don\'t know" .You dont hallucinate and you are fully reliable .\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = prompt | chat | parser 

In [6]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

Load Multiple files from Directory

In [7]:
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader(f'IIITDMJ', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [8]:
documents

[Document(metadata={'source': 'IIITDMJ\\Academic Guidelines_271017.pdf', 'page': 0}, page_content='1 \n \n \n \n \n \n \n \n \n \n \n \nAcademic Guidelines \n for  \nUndergraduate (UG) and Postgraduate (PG) \nProgrammes \nDumna Airport Road, P.O.: Khamaria, Jabalpur - 482 005, Madhya Pradesh, India  '),
 Document(metadata={'source': 'IIITDMJ\\Academic Guidelines_271017.pdf', 'page': 1}, page_content='2 \n \n1. The admission process \n1.1 Undergraduate Programmes (B.Tech./B.Des.):  \n1.1.1 The Institute shall offer four-year  \ni) Bachelors of Technology (B.Tech.) programme in  \n\uf0d8 Computer Science & Engineering (CSE),  \n\uf0d8 Electronics & Communications Engineering (ECE), and  \n\uf0d8 Mechanical Engineering (ME) \nii) Bachelor of Design (B.Des.) Programme \n1.1.2 Admissions to the B.Tech. programmes of the Institute are made once a year in July \nthrough the Centralized Admission Examination (Joint Entrance Examination (JEE \n(Main)). Counseling of the stu dents/ qualified can

Divide and Conquer

In [9]:

# Embed the chunks
text_splitter = CharacterTextSplitter(
    separator="\n",  # Split by lines
    chunk_size=300,  # Maximum chunk size
    chunk_overlap=200  # Overlap between chunks for context
)
split_docs = text_splitter.split_documents(documents)

In [10]:
split_docs[0]


Document(metadata={'source': 'IIITDMJ\\Academic Guidelines_271017.pdf', 'page': 0}, page_content='1 \n \n \n \n \n \n \n \n \n \n \n \nAcademic Guidelines \n for  \nUndergraduate (UG) and Postgraduate (PG) \nProgrammes \nDumna Airport Road, P.O.: Khamaria, Jabalpur - 482 005, Madhya Pradesh, India')

In [11]:
len(split_docs)

6309

Get Embeddings for OUR Documents

In [12]:
import pickle
import faiss
from langchain.vectorstores import FAISS

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\PRINCE SIDDHARTH\AppData\Local\Temp\ipykernel_21336\1248739731.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\PRINCE SIDDHARTH\OneDrive\Desktop\rag_proj2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
vector_store = FAISS.from_documents(split_docs, embeddings)

In [19]:
query = "PDPM IITDMJ  also known as?"
results = vector_store.similarity_search(query, k=2)
for result in results:
    print(result)

page_content='welcome and assure you to take this treasured journey together in the right earnest. 
PDPM IIITDMJ is one of the unique academic Institutes with regard to its curriculum and emphasize 
on project-oriented learning. I am happy to write that Indian as well as Japanese industries are' metadata={'source': 'IIITDMJ\\IIITDM Jabalpur.pdf', 'page': 13}
page_content='PDPM IIITDMJ is one of the unique academic Institutes with regard to its curriculum and emphasize 
on project-oriented learning. I am happy to write that Indian as well as Japanese industries are 
recognizing enhanced capabilities of our students due to the unique curriculum. The atmosphere on' metadata={'source': 'IIITDMJ\\IIITDM Jabalpur.pdf', 'page': 13}


In [20]:
from langchain.schema.runnable import Runnable, RunnableMap
from langchain.schema import BaseOutputParser

In [ ]:
from langchain.chains import LLMChain, RetrievalQA

from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS



# Step 3: Create an LLMChain using the prompt and the LLM
llm_chain = LLMChain(llm=chat, prompt=prompt)

# Step 4: Use a FAISS-based retriever
# Assuming `vector_store` is already created
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Step 5: Manually retrieve documents and process the context
query = "who is director?"


retrieved_docs = retriever.get_relevant_documents(query)

# Combine retrieved documents into the context string
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
input_data = {"context": context, "question": query}

# Step 6: Pass the input data to the LLMChain
result = llm_chain.run(input_data)

# Output the result
print("Answer:", result)

for doc in retrieved_docs:
    print("\nSource Document:", doc.page_content)



Answer: The Director of the Institute is Prof. Bhartendu Kumar Singh. You can reach him at his email address, director@iiitdmj.ac.in.

Source Document:  The Deputy Director  (to be nominated on appointment) 
 The Deans 
 The Heads of various disciplines and 
 The Registrar 
 
 
 
 
Building And Works Committee 
S. No.  Name Designation  
1.    Prof. Bhartendu Kumar  Singh 
Director 
PDPM-IIITDM Jabalpur 
director@iiitdmj.ac.in

Source Document: director@iiitdmj.ac.in 
2. Deputy Director To be nominated on appointment 
5.  Er . Sunil Trivedi 
SE (Electrical), MPPKVVCL 
secityjabalpur@gmail.com 
Member  
6.  Mrs. Swapnali Gadekar 
Acting Registrar and Officer-In-Charge 
Est 
PDPM IIITDM Jaba 
Secretary (Ex-Officio)

Source Document: governing the affairs of the Institute. 
The Director apart from being the Executive and Academic Head of the Institute is also a member of 
the Board with Registrar as its ex-officio Secretary. The day-to-day administration of the Institute is to


In [94]:
# Save the vector store
vector_store.save_local("vectorDB")
print("Vector store saved successfully.")

Vector store saved successfully.


In [95]:
from langchain.vectorstores import FAISS

def load_embeddings(embeddings, load_path="embedding"):
    vector_store = FAISS.load_local(load_path, embeddings)
    return vector_store